In [ ]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz, filtfilt
from pywt import swt, cwt
import scipy.misc
from scipy.signal import welch
import tensorflow as tf
import keras
from tensorflow import keras
import tensorflow.keras.backend as K
from keras.models import Sequential,Model
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input, AvgPool2D, Reshape, BatchNormalization, Bidirectional, LSTM
from keras.layers import Activation, GlobalMaxPool2D, SpatialDropout2D, GlobalAvgPool2D, SeparableConv1D
from keras.layers import AvgPool1D, Conv1D, GlobalMaxPool1D, MaxPool1D, SpatialDropout1D, GlobalAvgPool1D, Add, Concatenate
from keras.layers.convolutional import Conv1D, MaxPooling1D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.preprocessing import StandardScaler                                                      
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix 
from scipy import signal
import pickle as pkl
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_recall_fscore_support
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
gc.collect()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


129

In [ ]:
input_path='/content/drive/MyDrive/DREAMER_csv/'

In [ ]:
sampling_rate = 128
window_size = 1*128
skip = 64
channel_len = 14
classes = 2

bands = {'delta': [0.5/(sampling_rate/2), 4/(sampling_rate/2)], 'theta': [4/(sampling_rate/2), 8/(sampling_rate/2)], \
         'alpha': [8/(sampling_rate/2), 14/(sampling_rate/2)], 'beta': [14/(sampling_rate/2), 30/(sampling_rate/2)], \
         'gamma': [30/(sampling_rate/2), 50/(sampling_rate/2)]}

In [49]:
data = []
val_label = []
aro_label = []
dom_label = []

for person in range(2,3):
  print('Person No.' + str(person))
  
  # EEG files address
  address = input_path+'s'+str(person)
    

  # Loading Valence Label
  valence = pd.read_csv(address+ '/Label_1/'+ 's'+str(person)+'_val.csv' , header = None)
  valence = valence.values.ravel()

  # Loading Arousal Label
  arousal = pd.read_csv(address+ '/Label_1/'+ 's'+str(person)+'_aro.csv' , header = None)
  arousal = arousal.values.ravel()

  # Dominance values
  dominance = pd.read_csv(address+ '/Label_1/'+ 's'+str(person)+'_dom.csv' , header = None)
  dominance = dominance.values.ravel()
  

  # Preprocessing
  for i in range(1,19):
    eeg = pd.read_csv(address+'/s'+str(person)+'_v'+str(i)+'.csv', header=None)
    eeg = eeg.values
    print(eeg.shape)
    data.append(eeg)

Person No.2
(25472, 14)
(16768, 14)
(44544, 14)
(21248, 14)
(17408, 14)
(24320, 14)
(24576, 14)
(50432, 14)
(18560, 14)
(8576, 14)
(12288, 14)
(23168, 14)
(47104, 14)
(21760, 14)
(39424, 14)
(24960, 14)
(32768, 14)
(23808, 14)


In [51]:
data.max()

AttributeError: ignored

In [35]:
data = []
val_label = []
aro_label = []
dom_label = []

for person in range(2,3):
  print('Person No.' + str(person))
  
  # EEG files address
  address = input_path+'s'+str(person)
    

  # Loading Valence Label
  valence = pd.read_csv(address+ '/Label_1/'+ 's'+str(person)+'_val.csv' , header = None)
  valence = valence.values.ravel()

  # Loading Arousal Label
  arousal = pd.read_csv(address+ '/Label_1/'+ 's'+str(person)+'_aro.csv' , header = None)
  arousal = arousal.values.ravel()

  # Dominance values
  dominance = pd.read_csv(address+ '/Label_1/'+ 's'+str(person)+'_dom.csv' , header = None)
  dominance = dominance.values.ravel()
  

  # Preprocessing
  for i in range(1,19):
    eeg = pd.read_csv(address+'/s'+str(person)+'_v'+str(i)+'.csv', header=None)
    eeg = eeg.values
    scaler = StandardScaler().fit(eeg)
    scaled_eeg = scaler.transform(eeg)
    print(eeg.shape)
    print(scaled_eeg.shape)
    data.append(scaled_eeg)

Person No.2
(25472, 14)
(25472, 14)
(16768, 14)
(16768, 14)
(44544, 14)
(44544, 14)
(21248, 14)
(21248, 14)
(17408, 14)
(17408, 14)
(24320, 14)
(24320, 14)
(24576, 14)
(24576, 14)
(50432, 14)
(50432, 14)
(18560, 14)
(18560, 14)
(8576, 14)
(8576, 14)
(12288, 14)
(12288, 14)
(23168, 14)
(23168, 14)
(47104, 14)
(47104, 14)
(21760, 14)
(21760, 14)
(39424, 14)
(39424, 14)
(24960, 14)
(24960, 14)
(32768, 14)
(32768, 14)
(23808, 14)
(23808, 14)


In [36]:
data[9][100,:]

array([0.25126544, 0.04743814, 3.44931365, 2.23545353, 2.84924882,
       3.17253334, 2.93002361, 3.7676468 , 1.53397815, 3.39980826,
       3.40453303, 3.30043639, 1.15617252, 0.08833528])

In [40]:
data = []
val_label = []
aro_label = []
dom_label = []
window_size = 128
skip = 64

for person in range(2,3):
  print('Person No.' + str(person))
  
  # EEG files address
  address = input_path+'s'+str(person)
    

  # Loading Valence Label
  valence = pd.read_csv(address+ '/Label_1/'+ 's'+str(person)+'_val.csv' , header = None)
  valence = valence.values.ravel()

  # Loading Arousal Label
  arousal = pd.read_csv(address+ '/Label_1/'+ 's'+str(person)+'_aro.csv' , header = None)
  arousal = arousal.values.ravel()

  # Dominance values
  dominance = pd.read_csv(address+ '/Label_1/'+ 's'+str(person)+'_dom.csv' , header = None)
  dominance = dominance.values.ravel()
  
  # Assigning classes
  valence[valence<=3]=0
  valence[valence>3]=1  
  arousal[arousal<=3]=0
  arousal[arousal>3]=1
  dominance[dominance<=3]=0
  dominance[dominance>3]=1

  # Preprocessing
  for i in range(1,19):
    eeg = pd.read_csv(address+'/s'+str(person)+'_v'+str(i)+'.csv', header=None)
    eeg = eeg.values
    print(eeg.shape)
    
    # Segmenting into 1 seconds (1280 timesteps) windows with 0.5 seconds (256 timesteps) overlap
    start = 0
    while start+window_size <= eeg.shape[0]:
      data.append(eeg[start:start+window_size,:])
      val_label.append(valence[i-1])
      aro_label.append(arousal[i-1])
      dom_label.append(dominance[i-1])
      start = start+skip
  del eeg, valence, arousal, dominance

  data = np.array(data, dtype = np.float32)
  data = data.reshape([-1,128,14,1]) # Using 32 bit floating point value to save memory
  val_label = np.array(val_label, dtype = np.int8)
  aro_label = np.array(aro_label, dtype = np.int8)
  dom_label = np.array(dom_label, dtype = np.int8)

  print(data.shape)
  print(val_label.shape, val_label[val_label == 0].shape, val_label[val_label == 1].shape)
  print(aro_label.shape, aro_label[aro_label == 0].shape, aro_label[aro_label == 1].shape)
  print(dom_label.shape, dom_label[dom_label == 0].shape, dom_label[dom_label == 1].shape)

  val_label = np_utils.to_categorical(val_label)
  aro_label = np_utils.to_categorical(aro_label)
  dom_label = np_utils.to_categorical(dom_label) 

  print(val_label.shape)     
  print(aro_label.shape)
  print(dom_label.shape)

Person No.2
(25472, 14)
(16768, 14)
(44544, 14)
(21248, 14)
(17408, 14)
(24320, 14)
(24576, 14)
(50432, 14)
(18560, 14)
(8576, 14)
(12288, 14)
(23168, 14)
(47104, 14)
(21760, 14)
(39424, 14)
(24960, 14)
(32768, 14)
(23808, 14)
(7438, 128, 14, 1)
(7438,) (5073,) (2365,)
(7438,) (3121,) (4317,)
(7438,) (2871,) (4567,)
(7438, 2)
(7438, 2)
(7438, 2)


In [ ]:
data = []
val_label = []
aro_label = []
dom_label = []

for person in range(2,3):
  print('Person No.' + str(person))
  
  # EEG files address
  address = input_path+'s'+str(person)
    

  # Loading Valence Label
  valence = pd.read_csv(address+ '/Label_1/'+ 's'+str(person)+'_val.csv' , header = None)
  valence = valence.values.ravel()

  # Loading Arousal Label
  arousal = pd.read_csv(address+ '/Label_1/'+ 's'+str(person)+'_aro.csv' , header = None)
  arousal = arousal.values.ravel()

  # Dominance values
  dominance = pd.read_csv(address+ '/Label_1/'+ 's'+str(person)+'_dom.csv' , header = None)
  dominance = dominance.values.ravel()
  
  # Assigning classes
  valence[valence<=3]=0
  valence[valence>3]=1  
  arousal[arousal<=3]=0
  arousal[arousal>3]=1
  dominance[dominance<=3]=0
  dominance[dominance>3]=1

  # Preprocessing
  for i in range(1,19):
    eeg = pd.read_csv(address+'/s'+str(person)+'_v'+str(i)+'.csv', header=None)
    eeg = eeg.values
    print(eeg.shape)

    num, den = signal.butter(4, bands['alpha'], 'bandpass') # Butterworth filter of order N = 4
    band_signal = signal.filtfilt(num, den, eeg, axis=0)
    eeg = band_signal
    del band_signal, num, den
    
    scaler = StandardScaler().fit(eeg)
    scaled_eeg = scaler.transform(eeg)
    del eeg
    
    # Segmenting into 1 seconds (1280 timesteps) windows with 0.5 seconds (256 timesteps) overlap
    start = 0
    while start+window_size < scaled_eeg.shape[0]:
      data.append(scaled_eeg[start:start+window_size,:])
      val_label.append(valence[i-1])
      aro_label.append(arousal[i-1])
      dom_label.append(dominance[i-1])
      start = start+skip
    print(np.array(data).shape)
    del scaled_eeg

  data = np.array(data, dtype = np.float32) # Using 32 bit floating point value to save memory
  val_label = np.array(val_label, dtype = np.int8)
  aro_label = np.array(aro_label, dtype = np.int8)
  dom_label = np.array(dom_label, dtype = np.int8)

  print(data.shape)
  print(val_label.shape, val_label[val_label == 0].shape, val_label[val_label == 1].shape)
  print(aro_label.shape, aro_label[aro_label == 0].shape, aro_label[aro_label == 1].shape)
  print(dom_label.shape, dom_label[dom_label == 0].shape, dom_label[dom_label == 1].shape)

  val_label = np_utils.to_categorical(val_label)
  aro_label = np_utils.to_categorical(aro_label)
  dom_label = np_utils.to_categorical(dom_label)      


Person No.2
(25472, 14)
(396, 128, 14)
(16768, 14)
(656, 128, 14)
(44544, 14)
(1350, 128, 14)
(21248, 14)
(1680, 128, 14)
(17408, 14)
(1950, 128, 14)
(24320, 14)
(2328, 128, 14)
(24576, 14)
(2710, 128, 14)
(50432, 14)
(3496, 128, 14)
(18560, 14)
(3784, 128, 14)
(8576, 14)
(3916, 128, 14)
(12288, 14)
(4106, 128, 14)
(23168, 14)
(4466, 128, 14)
(47104, 14)
(5200, 128, 14)
(21760, 14)
(5538, 128, 14)
(39424, 14)
(6152, 128, 14)
(24960, 14)
(6540, 128, 14)
(32768, 14)
(7050, 128, 14)
(23808, 14)
(7420, 128, 14)
(7420, 128, 14)
(7420,) (5060,) (2360,)
(7420,) (3112,) (4308,)
(7420,) (2862,) (4558,)


In [ ]:
data = np.reshape(data,[-1,128,14,1])
print(data.shape)

(7420, 128, 14, 1)


In [20]:
data[0,0,:]

array([-0.03908394,  0.03489612, -0.03687253, -0.02460685,  0.11617757,
        0.14747638, -0.03002239,  0.00568334,  0.02283306, -0.06083841,
        0.00924395, -0.06204594, -0.08348725, -0.08252338], dtype=float32)

In [41]:
X_train, x_test, Y_train, y_test = train_test_split(data,val_label, test_size=0.2, random_state=4)
print(X_train.shape,x_test.shape,Y_train.shape,y_test.shape)

(5950, 128, 14, 1) (1488, 128, 14, 1) (5950, 2) (1488, 2)


In [42]:
batch_size = 96
epochs = 100
kfold = KFold(n_splits=10,shuffle=True)
gc.collect()

1266

# **Proposed Architecture**

In [43]:
def get_model() :
    input_shape = (data.shape[1],data.shape[2],1)
    model=Sequential()
    model.add(Conv2D(filters=16, kernel_size=(5,5), strides=(2,2), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    #model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=32, kernel_size=(7,7), strides=(2,2), padding='same'))
    model.add(BatchNormalization())
    #model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=64, kernel_size=(9,9), strides=(2,2), padding='same'))
    model.add(BatchNormalization())
    #model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256,activation='tanh'))
    model.add(Dropout(0.4))
    model.add(Dense(32,activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2,activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss=keras.losses.categorical_crossentropy, metrics=["accuracy"], optimizer=opt)
    return model


In [44]:
val_res = {'accuracy': [], 'f1_score': [], 'confusion_matrix': []}

foldNum=0
for train_index, val_index in kfold.split(X_train, Y_train):
  foldNum = foldNum + 1
  print("Results for fold",foldNum)
  x_train, x_val = X_train[train_index], X_train[val_index]
  y_train, y_val = Y_train[train_index], Y_train[val_index]
  model = get_model()
  model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(x_val, y_val),shuffle=True)
  acc = model.evaluate(x_test, y_test)
  print(acc)
  val_res['accuracy'].append(acc)
  pred = model.predict(x_test)
  val_res['f1_score'].append(f1_score(y_test.argmax(1), pred.argmax(1), average='macro'))
  val_res['confusion_matrix'].append(confusion_matrix(y_test.argmax(1), pred.argmax(1)))
  gc.collect() # Garbage collecter
  del x_train, x_val, y_train, y_val
  gc.collect()

Results for fold 1
Epoch 1/100
56/56 [==============================] - 2s 25ms/step - loss: 0.6723 - accuracy: 0.6644 - val_loss: 0.6244 - val_accuracy: 0.6908
Epoch 2/100
56/56 [==============================] - 1s 17ms/step - loss: 0.6367 - accuracy: 0.6820 - val_loss: 0.6198 - val_accuracy: 0.6908
Epoch 3/100
56/56 [==============================] - 1s 16ms/step - loss: 0.6375 - accuracy: 0.6820 - val_loss: 0.6209 - val_accuracy: 0.6908
Epoch 4/100
56/56 [==============================] - 1s 16ms/step - loss: 0.6326 - accuracy: 0.6825 - val_loss: 0.6188 - val_accuracy: 0.6908
Epoch 5/100
56/56 [==============================] - 1s 17ms/step - loss: 0.6281 - accuracy: 0.6824 - val_loss: 0.6225 - val_accuracy: 0.6908
Epoch 6/100
56/56 [==============================] - 1s 17ms/step - loss: 0.6326 - accuracy: 0.6825 - val_loss: 0.6240 - val_accuracy: 0.6908
Epoch 7/100
56/56 [==============================] - 1s 16ms/step - loss: 0.6293 - accuracy: 0.6825 - val_loss: 0.6196 - val_accu

KeyboardInterrupt: ignored

In [ ]:
for i in val_res['accuracy']:
  print(round(i[1]*100, 2)) # Rounding off to two decimal places
for i in val_res['confusion_matrix']:
  print(i)
for i in val_res['f1_score']:
  print(i*100)

60.92
60.78
59.3
61.73
59.43
[[251 163]
 [127 201]]
[[263 151]
 [140 188]]
[[251 163]
 [139 189]]
[[249 165]
 [119 209]]
[[245 169]
 [132 196]]
60.73816196648567
60.37684951893699
59.01302311969564
61.61351199714371
59.256279610261906


# **Arousal**

In [ ]:
X_train_a, x_test_a, Y_train_a, y_test_a = train_test_split(data,aro_label, test_size=0.2, random_state=4)
print(X_train_a.shape,x_test_a.shape,Y_train_a.shape,y_test_a.shape)

(2968, 128, 14, 1) (742, 128, 14, 1) (2968, 2) (742, 2)


In [ ]:
aro_res = {'accuracy': [], 'f1_score': [], 'confusion_matrix': []}

foldNum=0
for train_index, val_index in kfold.split(X_train_a, Y_train_a):
  foldNum = foldNum + 1
  print("Results for fold",foldNum)
  x_train_a, x_val_a = X_train_a[train_index], X_train_a[val_index]
  y_train_a, y_val_a = Y_train_a[train_index], Y_train_a[val_index]
  model = get_model()
  model.fit(x_train_a, y_train_a, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(x_val_a, y_val_a),shuffle=True)
  acc = model.evaluate(x_test_a, y_test_a)
  print(acc)
  aro_res['accuracy'].append(acc)
  pred = model.predict(x_test_a)
  aro_res['f1_score'].append(f1_score(y_test.argmax(1), pred.argmax(1), average='macro'))
  aro_res['confusion_matrix'].append(confusion_matrix(y_test.argmax(1), pred.argmax(1)))
  gc.collect() # Garbage collecter
  del x_train_a, x_val_a, y_train_a, y_val_a
  gc.collect()

Results for fold 1
Epoch 1/100
25/25 [==============================] - 1s 20ms/step - loss: 1.0352 - accuracy: 0.5518 - val_loss: 0.6846 - val_accuracy: 0.6667
Epoch 2/100
25/25 [==============================] - 0s 11ms/step - loss: 0.9456 - accuracy: 0.5965 - val_loss: 0.6774 - val_accuracy: 0.6465
Epoch 3/100
25/25 [==============================] - 0s 11ms/step - loss: 0.7920 - accuracy: 0.6264 - val_loss: 0.6322 - val_accuracy: 0.6751
Epoch 4/100
25/25 [==============================] - 0s 8ms/step - loss: 0.7667 - accuracy: 0.6340 - val_loss: 0.6630 - val_accuracy: 0.6414
Epoch 5/100
25/25 [==============================] - 0s 10ms/step - loss: 0.7395 - accuracy: 0.6306 - val_loss: 0.6380 - val_accuracy: 0.6717
Epoch 6/100
25/25 [==============================] - 0s 8ms/step - loss: 0.7227 - accuracy: 0.6276 - val_loss: 0.6378 - val_accuracy: 0.6751
Epoch 7/100
25/25 [==============================] - 0s 8ms/step - loss: 0.6806 - accuracy: 0.6386 - val_loss: 0.6182 - val_accurac

In [ ]:
for i in aro_res['accuracy']:
  print(round(i[1]*100, 2)) # Rounding off to two decimal places
for i in aro_res['confusion_matrix']:
  print(i)
for i in aro_res['f1_score']:
  print(i)

64.02
67.12
67.65
63.75
66.17
[[334  80]
 [228 100]]
[[349  65]
 [234  94]]
[[338  76]
 [209 119]]
[[345  69]
 [225 103]]
[[341  73]
 [231  97]]
0.5390635084548857
0.5430686309441672
0.5792504223014984
0.5566097560975609
0.5406209014557215


# **Dominance**

In [ ]:
X_train_d, x_test_d, Y_train_d, y_test_d = train_test_split(data,dom_label, test_size=0.2, random_state=4)
print(X_train_d.shape,x_test_d.shape,Y_train_d.shape,y_test_d.shape)

(2968, 128, 14, 1) (742, 128, 14, 1) (2968, 2) (742, 2)


In [ ]:
dom_res = {'accuracy': [], 'f1_score': [], 'confusion_matrix': []}

foldNum=0
for train_index, val_index in kfold.split(X_train_d, Y_train_d):
  foldNum = foldNum + 1
  print("Results for fold",foldNum)
  x_train_d, x_val_d = X_train_d[train_index], X_train_d[val_index]
  y_train_d, y_val_d = Y_train_d[train_index], Y_train_d[val_index]
  model = get_model()
  model.fit(x_train_d, y_train_d, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(x_val_d, y_val_d),shuffle=True)
  acc = model.evaluate(x_test_d, y_test_d)
  print(acc)
  dom_res['accuracy'].append(acc)
  pred = model.predict(x_test_d)
  dom_res['f1_score'].append(f1_score(y_test.argmax(1), pred.argmax(1), average='macro'))
  dom_res['confusion_matrix'].append(confusion_matrix(y_test.argmax(1), pred.argmax(1)))
  gc.collect() # Garbage collecter
  del x_train_d, x_val_d, y_train_d, y_val_d
  gc.collect()

Results for fold 1
Epoch 1/100
25/25 [==============================] - 1s 21ms/step - loss: 0.9523 - accuracy: 0.5270 - val_loss: 0.7332 - val_accuracy: 0.5758
Epoch 2/100
25/25 [==============================] - 0s 10ms/step - loss: 0.8467 - accuracy: 0.5556 - val_loss: 0.6961 - val_accuracy: 0.5993
Epoch 3/100
25/25 [==============================] - 0s 9ms/step - loss: 0.7685 - accuracy: 0.5813 - val_loss: 0.6746 - val_accuracy: 0.6431
Epoch 4/100
25/25 [==============================] - 0s 8ms/step - loss: 0.7330 - accuracy: 0.6238 - val_loss: 0.6503 - val_accuracy: 0.6448
Epoch 5/100
25/25 [==============================] - 0s 8ms/step - loss: 0.6771 - accuracy: 0.6361 - val_loss: 0.6211 - val_accuracy: 0.6532
Epoch 6/100
25/25 [==============================] - 0s 8ms/step - loss: 0.6627 - accuracy: 0.6445 - val_loss: 0.6197 - val_accuracy: 0.6532
Epoch 7/100
25/25 [==============================] - 0s 8ms/step - loss: 0.6214 - accuracy: 0.6685 - val_loss: 0.6457 - val_accuracy:

In [ ]:
for i in dom_res['accuracy']:
  print(round(i[1]*100, 2)) # Rounding off to two decimal places
for i in dom_res['confusion_matrix']:
  print(i)
for i in dom_res['f1_score']:
  print(i)

61.86
64.02
63.75
64.69
63.75
[[171 243]
 [116 212]]
[[142 272]
 [107 221]]
[[169 245]
 [104 224]]
[[160 254]
 [110 218]]
[[154 260]
 [123 205]]
0.5146907446577869
0.4833619009301463
0.5270510411130531
0.5064181286549708
0.481377392269186
